# Cryptocurrency Fundamental Analysis Tool

In [43]:
import json
import requests
import pandas as pd
import hvplot.pandas
import panel as pn
import holoviews as hv
import numpy as np

pn.extension(sizing_mode="stretch_width")

In [44]:

# Coin Metrics API endpoint url
url = 'https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?'

# Dates to get historical data for the last 3 years
start_date = pd.Timestamp("2018-01-01").isoformat()
end_date = pd.Timestamp.today().isoformat()

# Number of records to retrieve
page_size = 6000

# List of crypto assets to analyze
crypto_asset = ['btc', 'eth', 'doge', 'usdt']

def get_data(metric):
    
    # Save the JSON response as lists
    symbol = []
    time = []
    value = []

    # Loop through the selected tokens
    for token in crypto_asset:
        
        # Build URL with the corresponding token, metric, and dates
        get_url = f"{url}&assets={token}&metrics={metric}&start_time={start_date}&end_time={end_date}&page_size={page_size}"

        # Get the response from Coin Metrics API is JSON format
        response = requests.get(get_url).json()

        # Iterate through the response to add the relevant items to its corresponding list
        for item in response['data']:

            symbol.append(item['asset'])
            time.append(item['time'])
            value.append(item[metric])
            
    # Build the dataframe with the data
    data_df = pd.DataFrame(
    {'Token': symbol,
     'Date': time,
     'Value': value
    })
    
    # Format the date for easy reading 
    data_df['Date'] = pd.to_datetime(data_df['Date'], format="%Y/%m/%d").dt.date
    
    # Change the value type to 'float' to allow graphing
    data_df['Value'] = data_df['Value'].astype(float)
    
    # Format the DataFrame to allow xgraphing
    updated_data_df = data_df.pivot_table('Value', ['Date'], 'Token')
    
    return updated_data_df


In [45]:
# Create a dataframe for each metric by calling the get_data function and passing the relevant Coin Metric reference
prices = get_data('ReferenceRate')
active_addresses = get_data('AdrActCnt')
free_float_supply = get_data('SplyFF')
transaction_count = get_data('TxCnt')
market_cap_total = get_data('CapMrktCurUSD')
market_cap_real = get_data('CapRealUSD')
mk_value_percent = get_data('CapMVRVCur')
addresses_1 = get_data('AdrBalUSD1Cnt')
addresses_1m = get_data('AdrBalUSD1MCnt')


In [47]:
# Add a new column to label the DataFrame's data and identify each dataset after concatenation
prices['Label'] = "Prices"
active_addresses['Label'] = "Active Addresses"
free_float_supply['Label'] = "Free Float"
transaction_count['Label'] = 'Transaction Count'
market_cap_total['Label'] = "Market Cap, Total"
market_cap_real['Label'] = "Market Cap, Realized"
mk_value_percent['Label'] = "MVRV"
addresses_1['Label'] = "Addresses > $1"
addresses_1m['Label'] = "Addresses > $1M"

In [48]:
# List of DataFrames
get_data_df_list = [prices, active_addresses, free_float_supply, transaction_count, market_cap_real, market_cap_total, mk_value_percent, addresses_1, addresses_1m]

In [49]:
# Concatenate all DataFrames
get_data_df_all = pd.concat(get_data_df_list)

In [50]:
# Define the plots 
def get_plot(metric_df, ticker, window_size):
    df = get_data_df_all.loc[get_data_df_all['Label'] == metric_df][ticker]
    return df.hvplot.line('Date', grid=True, responsive=True, height=300, title=f"{ticker} {metric_df}", yformatter='%.0f')

In [51]:
# Create a dynamic plot with the information for all the tokens and metrics selected
interact = pn.interact(get_plot, metric_df=['Prices', 'Active Addresses', 'Free Float', 'Transaction Count', 'Market Cap, Total', 'Market Cap, Realized', 'MVRV', 'Addresses > $1', 'Addresses > $1M'], ticker=crypto_asset, window_size=(1, 21, 5))
interact

Column(sizing_mode='stretch_width')
    [0] Column(sizing_mode='stretch_width')
        [0] Select(name='metric_df', options=['Prices', 'Active Address...], sizing_mode='stretch_width', value='Prices')
        [1] Select(name='ticker', options=['btc', 'eth', ...], sizing_mode='stretch_width', value='btc')
        [2] IntSlider(end=21, name='window_size', sizing_mode='stretch_width', start=1, step=5, value=11)
    [1] Row(sizing_mode='stretch_width')
        [0] HoloViews(Curve, name='interactive03388', sizing_mode='stretch_width')

In [57]:
titles = ['Prices', 'Active Addresses', 'Free Float', 'Transaction Count', 'Market Cap, Total', 'Market Cap, Realized', 'MVRV', 'Addresses > $1', 'Addresses > $1M']
get_data_df_list = [prices, active_addresses, free_float_supply, transaction_count, market_cap_total, market_cap_real, mk_value_percent,addresses_1,addresses_1m]

latest_data_df = pd.DataFrame()

latest_data_df["Prices"] = get_data_df_all.loc[get_data_df_all['Label'] == "Prices"].iloc[-1]
latest_data_df["Active Addresses"] = get_data_df_all.loc[get_data_df_all['Label'] == "Active Addresses"].iloc[-1]
latest_data_df["Free Float"] = get_data_df_all.loc[get_data_df_all['Label'] == "Free Float"].iloc[-1]
latest_data_df["Transaction Count"] = get_data_df_all.loc[get_data_df_all['Label'] == "Transaction Count"].iloc[-1]
latest_data_df["Market Cap, Total"] = get_data_df_all.loc[get_data_df_all['Label'] == "Market Cap, Total"].iloc[-1]
latest_data_df["Market Cap, Realized"] = get_data_df_all.loc[get_data_df_all['Label'] == "Market Cap, Realized"].iloc[-1]
latest_data_df["MVRV"] = get_data_df_all.loc[get_data_df_all['Label'] == "MVRV"].iloc[-1]
latest_data_df["Addresses > $1"] = get_data_df_all.loc[get_data_df_all['Label'] == "Addresses > $1"].iloc[-1]
latest_data_df["Addresses > $1M"] = get_data_df_all.loc[get_data_df_all['Label'] == "Addresses > $1M"].iloc[-1]
latest_data_df = latest_data_df.T
latest_data_df

Token,btc,doge,eth,usdt,Label
Prices,47074.37071,0.339528,3310.727393,1.000789,Prices
Active Addresses,NaN,70944.0,542707.0,389.0,Active Addresses
Free Float,NaN,111884849452.091675,111681919.41163,1334796989.591662,Free Float
Transaction Count,NaN,28290.0,1253968.0,528.0,Transaction Count
"Market Cap, Total",NaN,41829773069.19767,369714840403.19458,1335346210.520165,"Market Cap, Total"
"Market Cap, Realized",380906526990.830811,18395964537.026577,193877932224.402802,1335279976.532763,"Market Cap, Realized"
MVRV,NaN,2.273856,1.906946,1.00005,MVRV
Addresses > $1,NaN,3271864.0,44488685.0,214859.0,Addresses > $1
Addresses > $1M,NaN,1596.0,16703.0,81.0,Addresses > $1M


In [53]:
latest_data_df.loc

In [54]:

interact = pn.interact(latest_data_df, ticker=crypto_asset, window_size=(1, 21, 5))

addresses = pn.Row(
    pn.Column(title_addresses, interact[0], sizing_mode="fixed", width=300),
    interact[1]
)

TypeError: unsupported callable